In [8]:
# import libraries
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, BatchNormalization, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing import image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tqdm import tqdm

In [9]:
# load data
gen_df = pd.read_csv('D:/data/Multi_Label_dataset/train.csv')

In [10]:
gen_df.shape

(7254, 27)

In [11]:
gen_df.isnull().sum()

Id             0
Genre          0
Action         0
Adventure      0
Animation      0
Biography      0
Comedy         0
Crime          0
Documentary    0
Drama          0
Family         0
Fantasy        0
History        0
Horror         0
Music          0
Musical        0
Mystery        0
N/A            0
News           0
Reality-TV     0
Romance        0
Sci-Fi         0
Short          0
Sport          0
Thriller       0
War            0
Western        0
dtype: int64

In [12]:
gen_df.head()

,Id,Genre,Action,Adventure,Animation,Biography,Comedy,Crime,Documentary,Drama,...,N/A,News,Reality-TV,Romance,Sci-Fi,Short,Sport,Thriller,War,Western
0,tt0086425,"['Comedy', 'Drama']",0,0,0,0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,tt0085549,"['Drama', 'Romance', 'Music']",0,0,0,0,0,0,0,1,...,0,0,0,1,0,0,0,0,0,0
2,tt0086465,['Comedy'],0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,tt0086567,"['Sci-Fi', 'Thriller']",0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0
4,tt0086034,"['Action', 'Adventure', 'Thriller']",1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


### image conversion to numpy arrays

In [13]:
width = 300
height = 300
X = []
for i in tqdm(range(gen_df.shape[0])):
    path = 'D:/data/Multi_Label_dataset/Images/'+gen_df['Id'][i]+'.jpg'
    img = image.load_img(path,target_size=(width, height, 3))
    img = image.img_to_array(img)
    img = img/255.0
    X.append(img)
X = np.array(X)

100%|█████████████████████████████████████████████████████████████████████████████| 7254/7254 [01:11<00:00, 101.63it/s]


MemoryError: Unable to allocate 7.30 GiB for an array with shape (7254, 300, 300, 3) and data type float32

In [16]:
#reducing array size 
lim_gen_df = gen_df.head(5000)
width = 300
height = 300
X = []
for i in tqdm(range(lim_gen_df.shape[0])):
    path = 'D:/data/Multi_Label_dataset/Images/'+lim_gen_df['Id'][i]+'.jpg'
    img = image.load_img(path,target_size=(width, height, 3))
    img = image.img_to_array(img)
    img = img/255.0
    X.append(img)
X = np.array(X)

100%|█████████████████████████████████████████████████████████████████████████████| 5000/5000 [00:30<00:00, 164.32it/s]


In [17]:
X.shape

(5000, 300, 300, 3)

In [18]:
y = lim_gen_df.drop(['Id','Genre'],axis=1)
y = y.to_numpy()
y.shape

(5000, 25)

In [19]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.1)

### Model

In [23]:
model = Sequential()
model.add(Conv2D(16,kernel_size=(3,3),activation='relu',input_shape=X_train[0].shape))
model.add(BatchNormalization())
model.add(MaxPool2D(2,2))
model.add(Dropout(0.3))
model.add(Conv2D(32,kernel_size=(3,3),activation = 'relu'))
model.add(MaxPool2D(2,2))
model.add(Dropout(0.3))
model.add(Conv2D(64,kernel_size=(3,3),activation = 'relu'))
model.add(MaxPool2D(2,2))
model.add(Dropout(0.4))

model.add(Flatten())

model.add(Dense(128,activation = 'relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(25,activation='sigmoid'))

In [24]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 298, 298, 16)      448       
                                                                 
 batch_normalization_2 (Batc  (None, 298, 298, 16)     64        
 hNormalization)                                                 
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 149, 149, 16)     0         
 2D)                                                             
                                                                 
 dropout_2 (Dropout)         (None, 149, 149, 16)      0         
                                                                 
 conv2d_3 (Conv2D)           (None, 147, 147, 32)      4640      
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 73, 73, 32)      

In [25]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [26]:
trans = model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

Epoch 1/10
141/141 [==============================] - 446s 3s/step - loss: 0.5924 - accuracy: 0.1744 - val_loss: 0.3125 - val_accuracy: 0.2120
Epoch 2/10
128/141 [==========================>...] - ETA: 34s - loss: 0.2740 - accuracy: 0.3091

KeyboardInterrupt: 